In [693]:
import os
import datetime
import glob
import pandas as pd

In [694]:
# Import helper functions
os.chdir('/Users/ishitagopal/Box/Projects/state_covid_policy/code/')
%run twitter_api_helper_functions.ipynb

In [726]:
# Set to main directory 
os.chdir('/Users/ishitagopal/Box/Projects/state_covid_policy/')

In [728]:
# Metadata round to use (example: _metadata_, _metadata_R2,  )
metaUse = '_metadata_'  # -- Needs to be assigned

fname1 = 'Data/*/*'+ metaUse + '*.csv'

# Import all files and combine them 
meta_files = glob.glob(fname1)

meta_li =[]
for file in meta_files:
    df = pd.read_csv(file, header = None)
    df['state'] = file.split('/')[1]
    meta_li.append(df)
    
meta = pd.concat(meta_li, axis=0, ignore_index=True, sort=False)
    

In [697]:
# Metadata column name types: -- Uncomment the relevant choice 

# Type 1
get_timeline_cols = ['screen_name', 'account_id', 'date_scraped','latest_tweet_id', 
                    'total_tweets', 'last_week_total_tweets', 'active_since', 
                    'active', 'state']

# # Type 2
# get_timeline_since_cols = ['screen_name', 'date_scraped', 'latest_tweet_id', 
#                            'total_tweets', 'active', 'state']


In [702]:
# Assign column names 
meta.columns = get_timeline_cols
meta

,screen_name,account_id,date_scraped,latest_tweet_id,total_tweets,last_week_total_tweets,active_since,active,state
0,LynnGreer2010,NaN,NaN,NaN,NaN,NaN,NaN,inactive,Alabama
1,Pdm2195,NaN,NaN,NaN,NaN,NaN,NaN,inactive,Alabama
2,andy_whitt1,2.87325e+09,2020-06-08,1.26943e+18,347,1.0,2014-11-12,active,Alabama
3,PronceyHouse7,1.08989e+09,2020-06-08,1.26985e+18,2754,44.0,2013-01-14,active,Alabama
4,RepTerriCollins,4.74388e+08,2020-06-08,1.2686e+18,767,1.0,2012-01-25,active,Alabama
...,...,...,...,...,...,...,...,...,...
8261,VoteSpringer,NaN,NaN,NaN,NaN,NaN,NaN,inactive,Washington
8262,debra_entenman,NaN,NaN,NaN,NaN,NaN,NaN,protected,Washington
8263,bhasegawa,3.67938e+07,8/6/20,1.24977e+18,205,0.0,4/30/09,active,Washington
8264,KarenKeiser,NaN,NaN,NaN,NaN,NaN,NaN,inactive,Washington


In [703]:
# Subset df to include only active accounts
active_meta = meta.loc[meta["active"]=='active',:].copy()

In [704]:
# Assign data types
active_meta['date_scraped'] = pd.to_datetime(active_meta['date_scraped'])
active_meta["account_id"] = active_meta["account_id"].astype(int)
active_meta["latest_tweet_id"] = active_meta["latest_tweet_id"].astype(int)

active_meta.dtypes

screen_name                       object
account_id                         int64
date_scraped              datetime64[ns]
latest_tweet_id                    int64
total_tweets                      object
last_week_total_tweets           float64
active_since                      object
active                            object
state                             object
dtype: object

In [705]:
# Number of active accounts v/s other categories
active_meta.active.value_counts()

active    3816
Name: active, dtype: int64

In [706]:
# Number of active accounts in each State 
active_meta.state.value_counts()

Pennsylvania      184
New_York          167
Massachusetts     157
Texas             156
Minnesota         148
New_Hampshire     138
Georgia           135
Florida           132
Maryland          128
Missouri          126
North_Carolina    118
Illinois          116
California        115
Virginia          105
South_Carolina    101
Ohio              100
Michigan           93
Kentucky           87
Connecticut        83
Colorado           83
Tennessee          83
New_Jersy          82
Arizona            78
Louisiana          78
Washington         77
Arkansas           77
Mississippi        74
Indiana            74
Kansas             71
Rhode_Island       68
Utah               63
New_Mexico         60
Nevada             56
Alabama            54
Oregon             53
Montana            47
Maine              44
Idaho              31
Vermont            31
North_Dakota       28
Hawaii             25
Alaska             24
South_Dakota       23
Nebraska           22
Delaware           21
Name: stat

In [707]:
# Maximum number tweets sent in a day 
max(active_meta.last_week_total_tweets)/7

197.28571428571428

In [708]:
# Total number of legislators who tweet more than a 100 times in a week
active_meta[active_meta.last_week_total_tweets > 100]['state'].value_counts().sum()

87

In [709]:
# Distribution of above by State 
active_meta[active_meta.last_week_total_tweets > 100]['state'].value_counts()

Arizona           9
Texas             7
Minnesota         7
New_Hampshire     7
New_Mexico        6
New_York          4
Colorado          4
California        3
Florida           3
Massachusetts     3
Kentucky          3
Tennessee         3
Delaware          2
Ohio              2
Georgia           2
Arkansas          2
North_Carolina    2
Alabama           2
Maryland          2
Pennsylvania      2
Nebraska          1
Nevada            1
Washington        1
Oregon            1
Indiana           1
South_Carolina    1
Vermont           1
Missouri          1
Virginia          1
Illinois          1
Mississippi       1
Rhode_Island      1
Name: state, dtype: int64

In [720]:
# Metadata round to create
metaCreate = 'metadata_R2'
'Data/*/*'+ metaUse + '*.csv'

In [711]:
# Identify States for which to collect tweets 
# Flags when 15 days have passed since the last download

date_today = datetime.datetime.today()
days_elapsed = date_today - active_meta.date_scraped

collect_states = active_meta.loc[days_elapsed.dt.days > 15, 'state'].unique()

collected_state = glob.glob('*')

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New_Hampshire', 'New_Jersy', 'New_Mexico', 'New_York',
       'North_Carolina', 'North_Dakota', 'Ohio', 'Oregon'], dtype=object)

In [723]:
state = 'Oregon'
# Navigate to State folder
path_state = ("/Users/ishitagopal/Box/Projects/state_covid_policy/Data/" + state)
os.chdir(path_state)

# Subset to create state specific dataframe
state_meta = active_meta.loc[active_meta['state']==state,:] 

# fname
fname2 = '%s_' % state + metaCreate +'.csv'


In [715]:
# Create metadata file  in the state directory if it doesnt exist 

if os.path.exists(fname)==True:
    print('File already exists')
    df = pd.read_csv(fmeta[0], index_col=0)
    handles = df.loc[df['active'].isnull(),:].index.to_list()
    since_id = state_meta.loc[state_meta['screen_name'].isin(handles), 'latest_tweet_id'].to_list()
    print(len(handles))
    
else:
    print('File Created')
    handles = state_meta['screen_name'].to_list()
    since_id = state_meta['latest_tweet_id'].to_list()
    df = pd.DataFrame(columns=cols,index=handles)
    df.to_csv(fname)
    print(len(handles))

File already exists
53


In [716]:
# Collect tweets since 
for i in range(0,len(handles)):    
    
    print('Index: %s, Handle: %s' % (i, handles[i]))
    
    # Read metadata csv 
    metadata = pd.read_csv(fname, index_col=0)

    # Download tweets
    results = get_timeline_since(handles[i], since_id[i])
    len(results['tweets'])
    metadata.loc[handles[i]] = results['metadata'][1:5]

    if len(results['tweets']) > 0:

        # Dump tweets to file
        date = today.strftime("%B%d") 
        file_name =  '%s/' % handles[i] + '%s_Tweets_%s' % (handles[i], date) + '.json'
        print(file_name)
        with open(file_name , 'w', encoding='utf8') as file:
            json.dump([tweet._json for tweet in results['tweets']], file)
            print("writen to file")

    metadata.to_csv(fname)

Index: 0, Handle: DavidBrockSmith
1
DavidBrockSmith/DavidBrockSmith_Tweets_August10.json
writen to file
Index: 1, Handle: RepCarlWilson
0
Index: 2, Handle: duane_stark
0
Index: 3, Handle: CedricHaydenOR
0
Index: 4, Handle: duckwilde
17
duckwilde/duckwilde_Tweets_August10.json
writen to file
Index: 5, Handle: Rep_NN
1
Rep_NN/Rep_NN_Tweets_August10.json
writen to file
Index: 6, Handle: juliefahey
15
juliefahey/juliefahey_Tweets_August10.json
writen to file
Index: 7, Handle: BoshartDavisAg
1
BoshartDavisAg/BoshartDavisAg_Tweets_August10.json
writen to file
Index: 8, Handle: DanRayfield
4
DanRayfield/DanRayfield_Tweets_August10.json
writen to file
Index: 9, Handle: EvansforOregon
0
Index: 10, Handle: Uncle_Mikey
0
Index: 11, Handle: NobleForOregon
protected
Index: 12, Handle: BillPostOregon
88
BillPostOregon/BillPostOregon_Tweets_August10.json
writen to file
Index: 13, Handle: RepNeronHD26
0
Index: 14, Handle: SollmanJaneen
19
SollmanJaneen/SollmanJaneen_Tweets_August10.json
writen to file

In [717]:
# Check the number of null rows -- should be zero 
metadata.active.isna().sum() 

0

In [718]:
# Number of active accounts
metadata.active.value_counts()


active       52
protected     1
Name: active, dtype: int64

In [719]:
# Number of accounts with 0 tweets
sum(metadata.total_tweets.value_counts()==0)

0